In [1]:
import sys
sys.path.append('../client')
from benchmark import format_f1_scores, f1_scores

In [2]:
def confusion(model, on_data : tuple[list[str], list[str]]):
    cm = defaultdict(lambda: defaultdict(int))
    for x, y in zip(*on_data):
        cm[y][model(x)] += 1
    return cm


In [3]:
import csv
from collections import defaultdict

import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split

SEED = 1337

with open('../data/atis/train.tsv', 'rt') as f:
    reader = csv.reader(f, delimiter='\t')
    data = list(reader)
    data_x = [row[0] for row in data]
    data_y = [row[1] for row in data]

data_x_train, data_x_val, data_y_train, data_y_val = map(np.array, train_test_split(data_x, data_y, test_size=0.2, random_state=SEED))

def val_accuracy(f):
    print(metrics.accuracy_score(data_y_val, f(data_x_val)))

data_x_words = sorted({w for x in data_x_train for w in x.split(' ')})

by_label = defaultdict(list)
for x, y in zip(data_x_train, data_y_train):
    by_label[y].append(x)
labels = set(by_label)

SCORE_DATA = { "train": [data_x_train, data_y_train], "val": [data_x_val, data_y_val]}

def print_f1_scores(model, data_key="train"):
    print(format_f1_scores(f1_scores(confusion(model, SCORE_DATA[data_key]))))

airfare_preditor = lambda s: (' price' in s) + (' cost ' in s) + (' fare' in s) + (' rate' in s) + (' ticket' in s) + ('how much' in s)

print_f1_scores(lambda s: 'airfare' if airfare_preditor(s) else 'flight')

  abbreviation (0, 0, 93): 0.0
  aircraft (0, 0, 58): 0.0
  airfare (299, 116, 20): 0.81
  airfare+flight_time (0, 0, 1): 0.0
  airline (0, 0, 118): 0.0
  airline+flight_no (0, 0, 1): 0.0
  airport (0, 0, 13): 0.0
  capacity (0, 0, 11): 0.0
  cheapest (0, 0, 1): 0.0
  city (0, 0, 15): 0.0
  distance (0, 0, 13): 0.0
  flight (2703, 589, 52): 0.89
  flight+airfare (0, 0, 14): 0.0
  flight_no (0, 0, 9): 0.0
  flight_time (0, 0, 44): 0.0
  ground_fare (0, 0, 14): 0.0
  ground_service (0, 0, 185): 0.0
  meal (0, 0, 2): 0.0
  quantity (0, 0, 37): 0.0
  restriction (0, 0, 4): 0.0


In [8]:
from transformers import pipeline

transformers_classifier_bart_large = pipeline("zero-shot-classification", model="facebook/bart-large-mnli") # 400m parameters
transformers_classifier_deberta_base = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli") # 200m parameters
transformers_classifier_minilm = pipeline("zero-shot-classification", model="MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli") # 100m parameters

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/428M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [11]:
BATCH_SIZE = 128

def confusion_batch(model, on_data : tuple[list[str], list[str]]):
    results = model(sequences=on_data[0], candidate_labels=sorted(labels), batch_size=BATCH_SIZE)
    cm = defaultdict(lambda: defaultdict(int))
    for y, y_ in zip(on_data[1], results):
        cm[y][y_] += 1
    return cm

tr_models = [transformers_classifier_minilm, transformers_classifier_deberta_base, transformers_classifier_bart_large]

for model in tr_models:
    print(format_f1_scores(f1_scores(confusion_batch(model, SCORE_DATA["train"]))))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  flight (0, 0, 2): 0.0
  ground_service (0, 0, 1): 0.0
  labels (0, 1, 0): 0.0
  scores (0, 1, 0): 0.0
  sequence (0, 1, 0): 0.0
  flight (0, 0, 2): 0.0
  ground_service (0, 0, 1): 0.0
  labels (0, 1, 0): 0.0
  scores (0, 1, 0): 0.0
  sequence (0, 1, 0): 0.0
  flight (0, 0, 2): 0.0
  ground_service (0, 0, 1): 0.0
  labels (0, 1, 0): 0.0
  scores (0, 1, 0): 0.0
  sequence (0, 1, 0): 0.0


In [12]:
transformers_classifier_bart_large("i want to fly to nyc", candidate_labels=sorted(labels))

{'sequence': 'i want to fly to nyc',
 'labels': ['flight',
  'airfare',
  'distance',
  'city',
  'flight+airfare',
  'abbreviation',
  'airfare+flight_time',
  'flight_time',
  'aircraft',
  'airline+flight_no',
  'flight_no',
  'capacity',
  'airport',
  'quantity',
  'airline',
  'restriction',
  'ground_fare',
  'ground_service',
  'meal',
  'cheapest'],
 'scores': [0.22391395270824432,
  0.13718608021736145,
  0.12011700868606567,
  0.11480510979890823,
  0.10111062228679657,
  0.0568232461810112,
  0.04757894575595856,
  0.03555750846862793,
  0.031680475920438766,
  0.022245541214942932,
  0.018322663381695747,
  0.017090201377868652,
  0.016434896737337112,
  0.013044649735093117,
  0.012528651393949986,
  0.011125268414616585,
  0.00860058143734932,
  0.005981918424367905,
  0.003734766272827983,
  0.0021179229952394962]}

In [13]:
def into_my_model(transformer_model):
    def as_model(xs):
        results = transformer_model(xs, batch_size=BATCH_SIZE, candidate_labels=sorted(labels))
        return [result['labels'][0] for result in results]
    return as_model

my_models = [into_my_model(model) for model in tr_models] 

In [16]:
my_models[0](['fly to nyc', 'take a taxi to penn station'])

['flight', 'capacity']

In [17]:
def confusion_batch(model, on_data : tuple[list[str], list[str]]):
    results = model(on_data[0])
    cm = defaultdict(lambda: defaultdict(int))
    for y, y_ in zip(on_data[1], results):
        cm[y][y_] += 1
    return cm
    
SCORE_DATA["2"] = ['fly to nyc', 'take a taxi to penn station'], ['flight', 'ground_service']

confusion_batch(my_models[0], SCORE_DATA["2"])

defaultdict(<function __main__.confusion_batch.<locals>.<lambda>()>,
            {'flight': defaultdict(int, {'flight': 1}),
             'ground_service': defaultdict(int, {'capacity': 1})})

In [18]:
for model in my_models:
    print(format_f1_scores(f1_scores(confusion_batch(model, SCORE_DATA["train"]))))

TypeError: string indices must be integers, not 'str'

Let's not make the data into an `np.array`

In [21]:
data_x_train, data_x_val, data_y_train, data_y_val = train_test_split(data_x, data_y, test_size=0.2, random_state=SEED)

SCORE_DATA["train"] = data_x_train, data_y_train
SCORE_DATA["val"] = data_x_val, data_y_val


In [22]:
for model in my_models:
    print(format_f1_scores(f1_scores(confusion_batch(model, SCORE_DATA["train"]))))

  abbreviation (56, 115, 37): 0.42
  aircraft (30, 7, 28): 0.63
  airfare (14, 3, 305): 0.083
  airfare+flight_time (0, 0, 1): 0.0
  airline (70, 59, 48): 0.57
  airline+flight_no (0, 1, 1): 0.0
  airport (12, 65, 1): 0.27
  capacity (6, 66, 5): 0.14
  cheapest (1, 149, 0): 0.013
  city (5, 3, 10): 0.43
  distance (7, 153, 6): 0.081
  flight (2259, 135, 496): 0.88
  flight+airfare (9, 122, 5): 0.12
  flight_no (0, 31, 9): 0.0
  flight_time (17, 15, 27): 0.45
  ground_fare (1, 118, 13): 0.015
  ground_service (8, 0, 177): 0.083
  meal (2, 45, 0): 0.082
  quantity (0, 46, 37): 0.0
  restriction (4, 73, 0): 0.099
  abbreviation (86, 87, 7): 0.65
  aircraft (10, 914, 48): 0.02
  airfare (229, 1340, 90): 0.24
  airfare+flight_time (0, 7, 1): 0.0
  airline (0, 65, 118): 0.0
  airline+flight_no (0, 16, 1): 0.0
  airport (7, 25, 6): 0.31
  capacity (2, 2, 9): 0.27
  cheapest (0, 3, 1): 0.0
  city (1, 2, 14): 0.11
  distance (9, 549, 4): 0.032
  flight (81, 17, 2674): 0.057
  flight+airfare (5,

In [23]:
len(SCORE_DATA["train"][0])

3707

In [24]:
import torch 
#  Check if a GPU is available and if not, use a CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [25]:
tr_models[0].device

device(type='cpu')

In [26]:
tr_model_names = [
    "MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli", # 100m parameters
    "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", # 200m parameters
    "facebook/bart-large-mnli", # 400m parameters
]
models = [pipeline("zero-shot-classification", model=name, device=device) for name in tr_model_names]

In [27]:
my_models = [into_my_model(model) for model in models]
my_models[0](SCORE_DATA["2"][0])

['flight', 'capacity']

In [28]:
from time import time
for model in my_models:
    start_time = time()
    print(format_f1_scores(f1_scores(confusion_batch(model, SCORE_DATA["train"]))))
    print(time() - start_time, "seconds elapsed")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  abbreviation (56, 115, 37): 0.42
  aircraft (30, 7, 28): 0.63
  airfare (14, 3, 305): 0.083
  airfare+flight_time (0, 0, 1): 0.0
  airline (70, 59, 48): 0.57
  airline+flight_no (0, 1, 1): 0.0
  airport (12, 65, 1): 0.27
  capacity (6, 66, 5): 0.14
  cheapest (1, 149, 0): 0.013
  city (5, 3, 10): 0.43
  distance (7, 153, 6): 0.081
  flight (2259, 135, 496): 0.88
  flight+airfare (9, 122, 5): 0.12
  flight_no (0, 31, 9): 0.0
  flight_time (17, 15, 27): 0.45
  ground_fare (1, 118, 13): 0.015
  ground_service (8, 0, 177): 0.083
  meal (2, 45, 0): 0.082
  quantity (0, 46, 37): 0.0
  restriction (4, 73, 0): 0.099
13.079231977462769 seconds elapsed
  abbreviation (86, 87, 7): 0.65
  aircraft (10, 914, 48): 0.02
  airfare (229, 1340, 90): 0.24
  airfare+flight_time (0, 7, 1): 0.0
  airline (0, 65, 118): 0.0
  airline+flight_no (0, 16, 1): 0.0
  airport (7, 25, 6): 0.31
  capacity (2, 2, 9): 0.27
  cheapest (0, 3, 1): 0.0
  city (1, 2, 14): 0.11
  distance (9, 549, 4): 0.032
  flight (81, 17

In [29]:
from time import time
for BATCH_SIZE in [16, 64, 256, 1024]:
    start_time = time()
    print(format_f1_scores(f1_scores(confusion_batch(my_models[0], SCORE_DATA["train"]))))
    print(time() - start_time, "seconds elapsed")

  abbreviation (56, 115, 37): 0.42
  aircraft (30, 7, 28): 0.63
  airfare (14, 3, 305): 0.083
  airfare+flight_time (0, 0, 1): 0.0
  airline (70, 59, 48): 0.57
  airline+flight_no (0, 1, 1): 0.0
  airport (12, 65, 1): 0.27
  capacity (6, 66, 5): 0.14
  cheapest (1, 149, 0): 0.013
  city (5, 3, 10): 0.43
  distance (7, 153, 6): 0.081
  flight (2259, 135, 496): 0.88
  flight+airfare (9, 122, 5): 0.12
  flight_no (0, 31, 9): 0.0
  flight_time (17, 15, 27): 0.45
  ground_fare (1, 118, 13): 0.015
  ground_service (8, 0, 177): 0.083
  meal (2, 45, 0): 0.082
  quantity (0, 46, 37): 0.0
  restriction (4, 73, 0): 0.099
21.843695878982544 seconds elapsed
  abbreviation (56, 115, 37): 0.42
  aircraft (30, 7, 28): 0.63
  airfare (14, 3, 305): 0.083
  airfare+flight_time (0, 0, 1): 0.0
  airline (70, 59, 48): 0.57
  airline+flight_no (0, 1, 1): 0.0
  airport (12, 65, 1): 0.27
  capacity (6, 66, 5): 0.14
  cheapest (1, 149, 0): 0.013
  city (5, 3, 10): 0.43
  distance (7, 153, 6): 0.081
  flight (22

In [30]:
from importlib import reload
reload("benchmarks")

TypeError: reload() argument must be a module

In [43]:
import benchmark
reload(benchmark)

<module 'benchmark' from '/home/we/Code/intent-classifier/docs/../client/benchmark.py'>

In [44]:
format_f1_scores = benchmark.format_f1_scores
f1_scores = benchmark.f1_scores

In [45]:
for model in my_models:
    print(format_f1_scores(f1_scores(confusion_batch(model, SCORE_DATA["val"]))))


/home/we/.virtualenvs/intent-classifier/lib/python3.12/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  abbreviation (8, 24, 7): 0.34
  aircraft (12, 2, 8): 0.71
  aircraft+flight+flight_no (0, 0, 1): 0.0
  airfare (6, 0, 78): 0.13
  airline (18, 15, 12): 0.57
  airline+flight_no (0, 0, 1): 0.0
  airport (5, 21, 0): 0.32
  capacity (3, 15, 2): 0.26
  cheapest (0, 45, 0): 0.0
  city (2, 3, 1): 0.5
  distance (1, 50, 6): 0.034
  flight (548, 38, 123): 0.87
  flight+airfare (2, 31, 1): 0.11
  flight_no (0, 8, 3): 0.0
  flight_time (2, 1, 6): 0.36
  ground_fare (0, 36, 3): 0.0
  ground_service (1, 0, 49): 0.039
  ground_service+ground_fare (0, 0, 1): 0.0
  meal (4, 7, 0): 0.53
  quantity (0, 9, 12): 0.0
  restriction (1, 9, 0): 0.18
  AVERAGE (21): 0.24
  abbreviation (12, 20, 3): 0.51
  aircraft (4, 231, 16): 0.031
  aircraft+flight+flight_no (0, 0, 1): 0.0
  airfare (63, 336, 21): 0.26
  airfare+flight_time (0, 2, 0): 0.0
  airline (0, 12, 30): 0.0
  airline+flight_no (0, 4, 1): 0.0
  airport (2, 9, 3): 0.25
  capacity (0, 0, 5): 0.0
  city (0, 2, 3): 0.0
  distance (6, 135, 1): 0.081
  